# Generating object masks from input prompts with SAM

In [ ]:
# %pip install segment-geospatial
# %pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html.
# %pip install xarray

In [2]:
import os
import leafmap
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff, split_raster, merge_rasters, raster_to_vector
from samgeo.text_sam import LangSAM
import xarray
import torch

# Plotting the Satellite imagery

In [1]:
image = "C:/Users/Data/Tifs/2018.tif"

In [3]:
m = leafmap.Map(center=[43.722069426835596, -79.37448883804542], zoom=18, height="1000px")
m.add_basemap("SATELLITE")
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m 

Map(center=[43.722069426835596, -79.37448883804542], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [5]:
# if m.user_roi_bounds() is not None:
#     bbox = m.user_roi_bounds()
# else:
#     bbox = [-122.2659, 37.8682, -122.2521, 37.8741]

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

In [4]:
# Define the input and output directories
results_dir = "C:/Users/Results/2021/"  # Directory to save the results

In [5]:
checkpoint = os.path.join(results_dir, 'sam_vit_h_4b8939.pth')

In [6]:
sam = SamGeo(
    model_type="vit_h",
    automatic = False,
    sam_kwargs=None,
    checkpoint=checkpoint,
    device = "cuda"
)

In [7]:
sam.set_image(image)

In [8]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [10]:
# This section is to split the image into tiles to be friendlier for model processing
# split_raster(image, out_dir=out_dir, tile_size=(1000, 1000), overlap=0)

# # List all image tiles
# tile_files = [f for f in os.listdir(tiles_dir) if os.path.isfile(os.path.join(tiles_dir, f))]

In [ ]:
# Process each tile
# for tile_file in tile_files:
#     tile_path = os.path.join(tiles_dir, tile_file)
#     output_path = os.path.join(masks_dir, f"mask_{tile_file}")
    
#     # Assuming the method to generate masks is called 'generate'
#     # You will need to adjust this according to the actual method's parameters and functionality
#     sam.generate(source=tile_path, output=output_path)
    
#     print(f"Processed {tile_path} and saved mask to {output_path}")

In [26]:
# output = "C:/Users/Data/2018_mask.tif"


In [31]:
# merge_rasters(masks_dir, output, input_pattern='*.tif', output_format='GTiff', output_nodata=None, output_options=['COMPRESS=DEFLATE'])
# sam.raster_to_vector(image = output, output= output_vector)

In [32]:
# sam = LangSAM()

In [ ]:
# Change prompt to the desired feature
# text_prompt = "tree"

In [36]:
# WDirectory to save the results
# tiles_dir = "C:/UsersResults/Tiles/"
# out_dir = "C:/Users/Results/Trees"  # Directory to save the results

In [ ]:
# sam.predict_batch(
#     images= tiles_dir,
#     out_dir= out_dir,
#     text_prompt=text_prompt,
#     box_threshold=0.6,
#     text_threshold=0.6,
#     mask_multiplier=255,
#     dtype="uint8",
#     merge=True,
#     verbose=True,
# )

In [42]:
# output_mask = "C:/Users/Results/Tree_mask.tif"
# output_vector = "C:/Users/Results/2018_mask.shp"

In [43]:
# merge_rasters(out_dir, output_mask, input_pattern='*.tif', output_format='GTiff', output_nodata=None, output_options=['COMPRESS=DEFLATE'])

In [44]:
# sam.raster_to_vector(image = output_mask, output= output_vector)

In [ ]:
# sam.show_masks(cmap="binary_r")

In [ ]:
# sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
# leafmap.image_comparison(
#     "satellite.tif",
#     "annotations.tif",
#     label1="Satellite Image",
#     label2="Image Segmentation",
# )

In [ ]:
# m.add_raster("annotations.tif", alpha=0.5, layer_name="Masks")
# m